In [2]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from linearmodels import IV2SLS, IVGMM, IVGMMCUE
from scipy.stats import f
from IPython.display import Latex

In [7]:
date_bound = '2012-05-01'

# 1 

a.	將value_factor_all.xlsx/ mom_factor_all.xlsx 中的TW_equity 、JP_equity 、US_equity和Global_equity_indice.xlsx中對應國家的return做迴歸（類似CAPM）無風險利率用Global_government_bonds.xlsx中應國家的return）


In [77]:
val = pd.read_excel('value_factor_all.xlsx')
val = val[val.Date>=date_bound].reset_index(drop = True)
val.Name = 'val'
mom = pd.read_excel('mom_factor_all.xlsx')
mom = mom[mom.Date>=date_bound].reset_index(drop = True)
mom.Name = 'mom'
couns = ['TW', 'JP', 'US']

equi = pd.ExcelFile('Global_equity_indice.xlsx')
rf_excel = pd.ExcelFile('Global_government_bonds.xlsx')
res_1a = pd.DataFrame(columns=['country', 'type', 'alpha', 't_value'])

for type in [val, mom]:
    for coun in couns:
        x_col = type.filter(like=coun, axis=1).columns
        xs = type[x_col]

        ys = pd.read_excel(equi, coun)
        ys =  ys[ys.Date>=date_bound].reset_index(drop = True)
        rf = pd.read_excel(rf_excel, coun)
        rf =  rf[rf.Date>=date_bound].reset_index(drop = True)
        for_model = pd.DataFrame({f'{coun}_{type.Name}' : xs.iloc[:, 0],  # 這邊不確定要不要扣rf，反正有先把rf抓出來了
                                  f'{coun}_indice' : ys['return']})

                
        x = for_model.iloc[:, 0]
        x = sm.add_constant(x)
        y = for_model.iloc[:, 1]

        model = sm.OLS(y, x)

        results = model.fit(cov_type='HAC', cov_kwds={'maxlags': 1}) # Newey-West std
        alpha = results.params['const']
        const_t_stat = results.tvalues['const']
        res_1a.loc[len(res_1a)] = [coun, type.Name, alpha, const_t_stat]
        # print(results.summary())

In [78]:
res_1a

,country,type,alpha,t_value
0,TW,val,0.004743,1.501164
1,JP,val,0.007160,1.832033
2,US,val,0.011288,1.532972
3,TW,mom,0.007036,2.151198
4,JP,mom,0.010304,2.556450
5,US,mom,0.015194,2.198686


b.	將value_factor_all.xlsx/ mom_factor_all.xlsx 中的Global_equity_indice和MSCI_world.xlsx中的mkt_return做迴歸
無風險利率用Global_government_bonds.xlsx中美國的return


In [81]:
val = pd.read_excel('value_factor_all.xlsx')
val = val[val.Date>=date_bound].reset_index(drop = True)[['Global_equity_indice']]
val.Name = 'val'
mom = pd.read_excel('mom_factor_all.xlsx')
mom = mom[mom.Date>=date_bound].reset_index(drop = True)[['Global_equity_indice']]
mom.Name = 'mom'

msciworld = pd.read_excel('MSCI_world.xlsx')
msciworld = msciworld[msciworld.Date>=date_bound].reset_index(drop = True)[['mkt_return']]
msciworld = msciworld.rename(columns = {'mkt_return' : 'msci_mkt_return'})

rf_excel = pd.ExcelFile('Global_government_bonds.xlsx')
res_1b = pd.DataFrame(columns=['type', 'alpha', 't_value'])

for type in [val, mom]:
    
    xs = type

    ys = msciworld

    rf = pd.read_excel(rf_excel, 'US')
    rf =  rf[rf.Date>=date_bound].reset_index(drop = True)
    for_model = pd.DataFrame({f'global_equity_{type.Name}' : xs.iloc[:, 0],  # 這邊不確定要不要扣rf，反正有先把rf抓出來了
                                'MSCI' : ys.iloc[:, 0]})

            
    x = for_model.iloc[:, 0]
    x = sm.add_constant(x)
    y = for_model.iloc[:, 1]

    model = sm.OLS(y, x)

    results = model.fit(cov_type='HAC', cov_kwds={'maxlags': 1}) # Newey-West std
    alpha = results.params['const']
    const_t_stat = results.tvalues['const']
    res_1b.loc[len(res_1b)] = [type.Name, alpha, const_t_stat]
    # print(results.summary())

In [82]:
res_1b

,type,alpha,t_value
0,val,0.006688,2.243308
1,mom,0.007178,2.254541


# 2

a.	將每個countryname_port_mom/value.xlsx中的不同Portfolio（一個檔案有三個，所以總共會跑九次）和MSCI_world.xlsx中的mkt_return、value_factor_all.xlsx中的value_everywhere、mom_factor_all.xlsx中的mom_everywhere做迴歸無風險利率用Global_government_bonds.xlsx中應國家的return）
這部分的回歸公式對應於論文中的(5)

b.	要做GRS test !! (好像是錯的)


$R^p_{it} - r^p_{f, t} = \alpha^p_i+\beta^p_iMKT_t+v^p_iVAL^{EVERYWHERE}_t+m^p_iMOM^{everywhere}_t+\epsilon^p_{it}$

In [29]:
# x
msciworld = pd.read_excel('MSCI_world.xlsx')
msciworld = msciworld[msciworld.Date>=date_bound].reset_index(drop = True)[['mkt_return']]
val = pd.read_excel('value_factor_all.xlsx')
val = val[val.Date>=date_bound].reset_index(drop = True)[['value_everywhere']]
mom = pd.read_excel('mom_factor_all.xlsx')
mom = mom[mom.Date>=date_bound].reset_index(drop = True)[['mom_everywhere']]

rf_excel = pd.ExcelFile('Global_government_bonds.xlsx')
res_2 = pd.DataFrame(columns=['country', 'type', 'rank', 'alpha', 't_value', 'grs_stat', 'grs_p_value'])
files = ['JP_port_mom.xlsx', 'JP_port_value.xlsx', 'TW_port_mom.xlsx', 'TW_port_value.xlsx', 'US_port_mom.xlsx', 'US_port_value.xlsx']

for file in files:
    xs = pd.read_excel(file)
    file = file.replace('.xlsx', '')
    coun = file.split('_')[0]
    type = file.split('_')[2]
    xs = xs[xs.Date>=date_bound].reset_index(drop = True)
    rf = pd.read_excel(rf_excel, coun)
    rf =  rf[rf.Date>=date_bound].reset_index(drop = True)['return']
    for i in range(1, 5):
        if i == 4:
            for_model = pd.DataFrame({'mkt' : msciworld.iloc[:, 0], 
                                    'val' : val.iloc[:, 0], 
                                    'mom' : val.iloc[:, 0], 
                                    'rp-rf' : xs.iloc[:, 3] - xs.iloc[:, 1]})
        else:
            for_model = pd.DataFrame({'mkt' : msciworld.iloc[:, 0], 
                                    'val' : val.iloc[:, 0], 
                                    'mom' : val.iloc[:, 0], 
                                    'rp-rf' : xs.iloc[:, i] - rf})
        X = sm.add_constant(for_model.iloc[:, :3])
        y = for_model.iloc[:, -1]
        model = sm.OLS(y, X)
        rank = f'P{i}'
        results = model.fit(cov_type='HAC', cov_kwds={'maxlags': 1}) # Newey-West std
        alpha = results.params['const']
        const_t_stat = results.tvalues['const']

        # grs test
        T = results.nobs
        N = 3  
        residuals = results.resid
        var_cov_single = np.cov(residuals, rowvar=False)

        if var_cov_single.ndim == 0:        # 檢查 var_cov_single 的形狀
            var_cov_single = np.array([[var_cov_single]])
        grs_stat = ((T - N - 1) / N) * ((alpha ** 2) / np.dot(alpha.T, np.linalg.inv(var_cov_single)).dot(alpha))[0][0]
        df1 = N
        df2 = T - N - 1
        grs_p_value = 1 - f.cdf(grs_stat, df1, df2)
        rank = 'P3-P1' if i == 4 else f'P{i}'
        res_2.loc[len(res_2)] = [coun, type, rank, alpha, const_t_stat, grs_stat, grs_p_value]


In [30]:
res_2

,country,type,rank,alpha,t_value,grs_stat,grs_p_value
0,JP,mom,P1,-0.002871,-0.415974,0.218319,0.883552
1,JP,mom,P2,0.007966,1.258962,0.205216,0.892643
2,JP,mom,P3,0.007993,1.308297,0.223812,0.879714
3,JP,mom,P3-P1,0.010865,1.292263,0.402431,0.751478
4,JP,value,P1,-0.002519,-0.426795,0.173131,0.914409
5,JP,value,P2,-0.002146,-0.405227,0.194825,0.899776
6,JP,value,P3,0.012846,1.713637,0.253935,0.858423
7,JP,value,P3-P1,0.015365,2.035797,0.280933,0.839090
8,TW,mom,P1,-0.000735,-0.184954,0.112548,0.952640
9,TW,mom,P2,-0.001265,-0.395121,0.059815,0.980754


# 3

a.	將countryname_port_mom/value.xlsx中的不同Portfolio用FF 3 factors和5 factors再跑一次，一樣要做GRS test !!
還沒做

$R^p_{it} - r^p_{f, t} = \alpha^p_i+\beta^p_iMKT_t+v^p_iVAL^{EVERYWHERE}_t+m^p_iMOM^{everywhere}_t+\epsilon^p_{it}$

In [63]:
xs

,Date,Top Third Portfolio,Middle Third Portfolio,Bottom Third Portfolio
0,2012-05-01,-0.012213,-0.179846,0.005956
1,2012-06-01,-0.059719,0.161658,0.022321
2,2012-07-01,0.017403,-0.065124,-0.079418
3,2012-08-01,0.086253,0.052979,0.056578
4,2012-09-01,-0.014716,-0.012579,-0.001285
...,...,...,...,...
123,2022-08-01,0.050594,0.014108,-0.003848
124,2022-09-01,-0.073855,-0.076324,-0.124759
125,2022-10-01,0.029335,0.046797,0.290147
126,2022-11-01,-0.034630,-0.051526,0.074484


In [65]:
# x
ff3 = pd.read_csv('F-F_Research_Data_3_Factors.csv')
ff3 = ff3[pd.to_datetime(ff3['date']) >= pd.to_datetime(date_bound)].reset_index(drop = True)
ff5 = pd.read_csv('F-F_Research_Data_5_Factors_2x3.csv')
ff5 = ff5[pd.to_datetime(ff5['date']) >= pd.to_datetime(date_bound)].reset_index(drop = True)

rf_excel = pd.ExcelFile('Global_government_bonds.xlsx')
res_2 = pd.DataFrame(columns=['country', 'type', 'rank', 'alpha', 't_value', 'grs_stat', 'grs_p_value'])
files = ['JP_port_mom.xlsx', 'JP_port_value.xlsx', 'TW_port_mom.xlsx', 'TW_port_value.xlsx', 'US_port_mom.xlsx', 'US_port_value.xlsx']

for file in files:
    xs = pd.read_excel(file)
    file = file.replace('.xlsx', '')
    coun = file.split('_')[0]
    type = file.split('_')[2]
    xs = xs[xs.Date>=date_bound]
    xs = xs[xs.Date<=pd.to_datetime('2022-12-31')].reset_index(drop = True)
    rf = pd.read_excel(rf_excel, coun)
    rf =  rf[rf.Date>=date_bound]
    rf = rf[rf.Date<=pd.to_datetime('2022-12-31')].reset_index(drop = True)['return']
    for i in range(1, 5):
        if i == 4:
            y = xs.iloc[:, 3] - xs.iloc[:, 1]
        else:
            y = xs.iloc[:, i] - rf

        X = sm.add_constant(ff3.iloc[:, 1:4])
        # y = for_model.iloc[:, -1]
        model = sm.OLS(y, X)
        rank = f'P{i}'
        results = model.fit(cov_type='HAC', cov_kwds={'maxlags': 1}) # Newey-West std
        alpha = results.params['const']
        const_t_stat = results.tvalues['const']

        # grs test
        T = results.nobs
        N = 3  
        residuals = results.resid
        var_cov_single = np.cov(residuals, rowvar=False)

        if var_cov_single.ndim == 0:        # 檢查 var_cov_single 的形狀
            var_cov_single = np.array([[var_cov_single]])
        grs_stat = ((T - N - 1) / N) * ((alpha ** 2) / np.dot(alpha.T, np.linalg.inv(var_cov_single)).dot(alpha))[0][0]
        df1 = N
        df2 = T - N - 1
        grs_p_value = 1 - f.cdf(grs_stat, df1, df2)
        rank = 'P3-P1' if i == 4 else f'P{i}'
        res_2.loc[len(res_2)] = [coun, type, rank, alpha, const_t_stat, grs_stat, grs_p_value]


In [66]:
res_2

,country,type,rank,alpha,t_value,grs_stat,grs_p_value
0,JP,mom,P1,-0.004026,-0.644559,0.193961,0.900346
1,JP,mom,P2,0.007080,1.334095,0.171310,0.915600
2,JP,mom,P3,0.006612,1.192441,0.187215,0.904932
3,JP,mom,P3-P1,0.010638,1.331279,0.362049,0.780513
4,JP,value,P1,-0.005084,-0.974018,0.153718,0.927125
5,JP,value,P2,-0.001701,-0.340686,0.178386,0.910880
6,JP,value,P3,0.012020,1.891367,0.208785,0.890158
7,JP,value,P3-P1,0.017104,2.425329,0.246377,0.863782
8,TW,mom,P1,-0.003598,-0.895702,0.108496,0.954995
9,TW,mom,P2,-0.001335,-0.406513,0.054859,0.983012
